In [ ]:
import monai.metrics as metrics
import torch

from src.data_modules.segmentation_data_module import CbisDdsmDataModuleSegmentation
from src.models.segmentation_model import SegmentationModel
from tqdm import tqdm

In [54]:
datamodule = CbisDdsmDataModuleSegmentation(
    root_dir="data/cbis-ddsm-segme",
    tumor_type="mass",
    batch_size=8,
    num_workers=4,
)

In [55]:
checkpoint = "models/mass-segmentation.ckpt"
model = SegmentationModel.load_from_checkpoint(checkpoint, map_location="cpu")
model = model.eval()

### Dice Score and Intersection over Union (IoU)

#### Dice Score

The Dice Score, also known as the Dice Similarity Coefficient (DSC), is a metric used to evaluate the similarity between two sets, commonly used in image segmentation tasks. It measures the overlap between the predicted segmentation and the ground truth. The Dice Score is defined as:

$$\text{Dice Score} = \frac{2 \cdot |A \cap B|}{|A| + |B|}$$

Where:
- $A$ is the set of predicted pixels (or regions).
- $B$ is the set of ground truth pixels (or regions).

The Dice Score ranges from 0 to 1:
- A score of 1 indicates perfect overlap.
- A score of 0 indicates no overlap.

#### Intersection over Union (IoU)

Intersection over Union (IoU), also known as the Jaccard Index, is another metric used to evaluate the accuracy of image segmentation. It measures the ratio of the intersection to the union of the predicted segmentation and the ground truth. IoU is defined as:

$$\text{IoU} = \frac{|A \cap B|}{|A \cup B|}$$

Where:
- $A$ is the set of predicted pixels (or regions).
- $B$ is the set of ground truth pixels (or regions).

The IoU also ranges from 0 to 1:
- A score of 1 indicates perfect overlap.
- A score of 0 indicates no overlap.

#### Comparison

Both Dice Score and IoU are widely used in segmentation tasks, but they differ slightly in their sensitivity to false positives and false negatives. The Dice Score tends to give more weight to the overlap, while IoU provides a stricter measure of similarity. These metrics are often used together to provide a comprehensive evaluation of segmentation performance.

In [61]:
test_dataloader = datamodule.test_dataloader()

# Initialize metrics
dice_metric = metrics.DiceMetric(reduction="mean")
iou_metric = metrics.MeanIoU(reduction="mean")

# Evaluate metrics
for images, masks in tqdm(test_dataloader):
    with torch.no_grad():
        predictions = model(images)

    predictions = torch.argmax(predictions, dim=1)

    y_pred = predictions.unsqueeze(1)
    y_true = torch.argmax(masks, dim=1).unsqueeze(1)

    dice_metric(y_pred, y_true)
    iou_metric(y_pred, y_true)

# Compute Dice Score
dice_score = dice_metric.aggregate().item()

# Compute IoU Score
iou_score = iou_metric.aggregate().item()

print("Dice Score:", dice_score)
print("IoU Score:", iou_score)

100%|██████████| 231/231 [02:41<00:00,  1.43it/s]

Dice Score: 0.7969831824302673
IoU Score: 0.6748219132423401


### Model Evaluation

#### Performance Metrics

The evaluation of our segmentation model is based on the above described metrics. These metrics provide insights into the model's ability to accurately segment mass tumors.

- **Dice Score**: 0.8
    The Dice Score indicates a high degree of overlap between the predicted segmentation and the ground truth. A score of 0.8 suggests that the model performs well in capturing the target regions with minimal false positives and false negatives.

- **IoU**: 0.7
    The IoU score of 0.7 demonstrates that the model achieves a strong balance between precision and recall, effectively identifying the intersection of predicted and ground truth regions relative to their union.

#### Conclusion

The model's performance, with a Dice Score of 0.8 and an IoU of 0.7, reflects a robust segmentation capability. These results indicate that the model is well-suited for the task and can be considered reliable for practical applications. However, further fine-tuning or additional data may help improve the performance even further and the oversight of a medical professional is highly recommended.